# KGCN 논문 리뷰 & 코드 작성
**KGCN: Simplifying and Powering Graph Convolution Network for Recommendation**  
*Hongwei Wang et al. (2019)*  
🔗 [논문 링크](https://arxiv.org/abs/1904.12575)

## 3.1 Problem Formulation

*목표 : 사용자 u가 아이템 v에 관심 있을지를 예측하는 함수*
$$\hat{y}_{uv} = F(u,v | Θ, Y, G)$$
- Y : 사용자-아이템 상호작용 행렬 (예: 클릭, 평가)
- G : 지식 그래프 (KG), 삼중항(triple : head, relation, tail)의 집합

## 3.2 KGCN Layer (모델 구성 요소)

*1. 관계 중요도 계산 (사용자 u, 관계 r)*
$$\pi_{ur} = g(u,r)$$

*2. 이웃 노드 집계 (attention 가중치 포함)*
$$v_u^{N(v)} = \sigma_{e\in{N(v)}}\tilde{\pi}_{ur}⋅e$$
$$\tilde{\pi}_{ur}=\frac{exp(\pi_{ur})}{\sum\nolimits_{e'}exp(\pi_{ur'})}$$

*3. Aggregation 방식 3가지*
- Sum : $ReLU(W(v+v_u^{N(v)})+b)$
- Concat : $ReLU(W[v;v_u^{N(v)}]+b)$
- Neighbor : $ReLU(Wv_u^{N(v)}+b)$

## 3.3 Learning Algorithm (학습 알고리즘)

*반복 구조*
KGCN은 여러 계층(hop)으로 구성되어 '0-hop → 1-hop → ...'와 같은 형식으로 이웃 정보를 반복적으로 전파 및 집계
$$\hat{y}_{uv}=f(u, v_u^{(h)})$$

*학습 손실 함수*
- Cross Entropy + Negative Sampling + L2 정규화 포함
$$L = \sum_{u}\begin{bmatrix}\sum_{v:y_{uv}=1}J(y_{uv}, \hat{y}_{uv})-\sum_{i=1}^{T_u}\mathbb{E}_{vi~P(v)}J(0,\hat{y}_(uvi))\end{bmatrix}+\lambda||F||_2^2$$

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import argparse
import matplotlib.pyplot as plt

In [2]:
device = (torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu"))
print("Using device:", device)

Using device: mps


## Book - Crossing 모델 설정 (Hyperparameter)

|항목|설정|
|:---:|:---:|
임베딩 차원 | 64
학습률 | 0.0002
Optimizer | Adam
정규화 계수 (L2) | 2e-5
Negative Sampling | 1:1 비율로 샘플링
배치 사이즈 | 256
학습 Epoch | 최대 1000 (일반적으로 200~400)
레이어 수 (Receptive Field Depth, H) | 1
이웃 샘플링 수 (K) | 8
초기화 방식 | Xavier Uniform (명시는 없지만 일반적인 초기화 방식으로 추정됨)

In [3]:
config = {
    'item2id_path': 'data/item_index2entity_id.txt',
    'kg_path': 'data/kg.txt',
    'rating_path': 'data/user_artists.dat',
    'rating_sep': '\t',
    'threshold': 0.0,
    
    'embedding_dim': 16,
    'n_epochs': 20,
    'neighbor_sample_size': 8,
    'n_iter': 1,
    'batch_size': 256,
    'l2_weight': 1e-4,
    'lr': 5e-4,
    'train_ratio': 0.8,
    'aggregator': 'sum',
    'device': 'mps'
}

In [4]:
class KGDataLoader:
    def __init__(self, cfg):
        self.cfg = cfg  # 설정 저장

        # 파일 로드
        df_item2id = pd.read_csv(self.cfg['item2id_path'], sep='\t', header=None, names=['item', 'id'])  # 아이템→엔티티 매핑
        df_kg = pd.read_csv(self.cfg['kg_path'], sep='\t', header=None, names=['head', 'relation', 'tail'])  # 지식 그래프
        df_rating = pd.read_csv(self.cfg['rating_path'], sep=self.cfg['rating_sep'], 
                                names=['userID', 'itemID', 'rating'], skiprows=1)  # 사용자-아이템 평점

        # 매핑에 존재하는 아이템만 필터링
        df_rating = df_rating[df_rating['itemID'].isin(df_item2id['item'])]
        df_rating.reset_index(inplace=True, drop=True)

        # 데이터프레임 저장
        self.df_item2id = df_item2id
        self.df_kg = df_kg
        self.df_rating = df_rating

        # 인코더 준비
        self.user_encoder = LabelEncoder()
        self.entity_encoder = LabelEncoder()
        self.relation_encoder = LabelEncoder()

        # ID 인코딩 실행
        self._encoding()

    def _encoding(self): # userID, entityID, relation을 정수로 인코딩
        self.user_encoder.fit(self.df_rating['userID'])
        self.entity_encoder.fit(pd.concat([self.df_item2id['id'], self.df_kg['head'], self.df_kg['tail']]))
        self.relation_encoder.fit(self.df_kg['relation'])

        # 변환 적용
        self.df_kg['head'] = self.entity_encoder.transform(self.df_kg['head'])
        self.df_kg['tail'] = self.entity_encoder.transform(self.df_kg['tail'])
        self.df_kg['relation'] = self.relation_encoder.transform(self.df_kg['relation'])

    def _build_dataset(self): # positive + negative 샘플 생성하여 학습용 데이터셋 구성
        print('Build dataset dataframe ...', end=' ')
        df_dataset = pd.DataFrame()

        # 사용자 ID 인코딩
        df_dataset['userID'] = self.user_encoder.transform(self.df_rating['userID'])

        # 아이템 문자열을 entity ID로 매핑
        item2id_dict = dict(zip(self.df_item2id['item'], self.df_item2id['id']))
        self.df_rating['itemID'] = self.df_rating['itemID'].apply(lambda x: item2id_dict[x])

        # 엔티티 인코딩
        df_dataset['itemID'] = self.entity_encoder.transform(self.df_rating['itemID'])

        # 평점을 임계값 기준으로 이진 라벨 생성
        df_dataset['label'] = self.df_rating['rating'].apply(lambda x: 0 if x < self.cfg['threshold'] else 1)

        # positive만 사용
        df_dataset = df_dataset[df_dataset['label'] == 1]

        # 전체 엔티티 집합에서 negative 샘플링
        full_item_set = set(range(len(self.entity_encoder.classes_)))
        user_list, item_list, label_list = [], [], []
        for user, group in df_dataset.groupby('userID'):
            item_set = set(group['itemID'])  # positive 아이템 집합
            negative_set = full_item_set - item_set  # negative 후보
            negative_sampled = random.sample(list(negative_set), len(item_set))  # 같은 수 만큼 샘플링

            # negative 샘플 저장
            user_list.extend([user] * len(negative_sampled))
            item_list.extend(negative_sampled)
            label_list.extend([0] * len(negative_sampled))

        # negative를 데이터프레임으로 만들고 결합
        negative = pd.DataFrame({'userID': user_list, 'itemID': item_list, 'label': label_list})
        df_dataset = pd.concat([df_dataset, negative])

        # 셔플 및 인덱스 초기화
        df_dataset = df_dataset.sample(frac=1, replace=False, random_state=999)
        df_dataset.reset_index(inplace=True, drop=True)

        print('Done')
        return df_dataset

    def _construct_kg(self): # 양방향 knowledge graph 딕셔너리 구성
        print('Construct knowledge graph ...', end=' ')
        kg = dict()
        for i in range(len(self.df_kg)):
            head = self.df_kg.iloc[i]['head']
            relation = self.df_kg.iloc[i]['relation']
            tail = self.df_kg.iloc[i]['tail']

            # head → tail
            if head in kg:
                kg[head].append((relation, tail))
            else:
                kg[head] = [(relation, tail)]

            # tail → head (양방향)
            if tail in kg:
                kg[tail].append((relation, head))
            else:
                kg[tail] = [(relation, head)]

        print('Done')
        return kg

    def load_dataset(self): # 학습용 사용자-아이템 데이터셋 로드
        return self._build_dataset()

    def load_kg(self): # 지식 그래프 딕셔너리 로드
        return self._construct_kg()

    def get_encoders(self): # user, entity, relation 인코더 반환
        return (self.user_encoder, self.entity_encoder, self.relation_encoder)

    def get_num(self): # 각 인코더의 클래스 수 반환 (모델 입력 차원 계산용)
        return (len(self.user_encoder.classes_), 
                len(self.entity_encoder.classes_), 
                len(self.relation_encoder.classes_))

In [5]:
# 설정 정보를 담은 딕셔너리를 사용해 KGDataLoader 인스턴스를 생성
data_loader = KGDataLoader(cfg=config)

# 학습용 사용자-아이템 데이터셋을 생성 (positive + negative 샘플 포함)
df_dataset = data_loader.load_dataset()

# 레이블(positive: 1, negative: 0) 분포 출력
print("레이블 분포:")
print(df_dataset['label'].value_counts())

# 전체 데이터 중 레이블 비율 (%)로 출력
print("\n레이블 비율 (%):")
print(df_dataset['label'].value_counts(normalize=True) * 100)

# 데이터 타입 확인
print(df_dataset.dtypes)


Build dataset dataframe ... Done
레이블 분포:
label
1    21173
0    21173
Name: count, dtype: int64

레이블 비율 (%):
label
1    50.0
0    50.0
Name: proportion, dtype: float64
userID    int64
itemID    int64
label     int64
dtype: object


In [6]:
class Aggregator(torch.nn.Module):
    
    def __init__(self, batch_size, dim, aggregator):
        super(Aggregator, self).__init__()
        self.batch_size = batch_size  # 미니배치 크기
        self.dim = dim                # 임베딩 차원
        self.aggregator = aggregator  # aggregation 방식: sum / concat / neighbor-only

        # aggregator 종류에 따라 파라미터 설정
        if aggregator == 'concat':
            # self 임베딩 + 이웃 임베딩을 concat하면 차원이 2배가 되므로
            self.weights = torch.nn.Linear(2 * dim, dim, bias=True)
        else:
            # sum or neighbor-only인 경우는 차원 그대로
            self.weights = torch.nn.Linear(dim, dim, bias=True)
        
    def forward(self, self_vectors, neighbor_vectors, neighbor_relations, user_embeddings, act):
        # self_vectors: [batch_size, -1, dim]   → 현재 엔티티 (노드) 임베딩
        # neighbor_vectors: [batch_size, -1, n_neighbor, dim] → 이웃 엔티티 임베딩
        # neighbor_relations: [batch_size, -1, n_neighbor, dim] → 이웃 관계 임베딩
        # user_embeddings: [batch_size, dim] → 사용자 임베딩 (user-aware attention에 사용)
        # act: 활성화 함수 (예: torch.relu)
        batch_size = user_embeddings.size(0)

        # batch_size가 변할 수 있어 동적으로 업데이트
        if batch_size != self.batch_size:
            self.batch_size = batch_size

        # 이웃 임베딩을 user-aware 방식으로 합치는 과정
        neighbors_agg = self._mix_neighbor_vectors(neighbor_vectors, neighbor_relations, user_embeddings)

        # aggregation 방식에 따라 self vector와 합치는 방식 달라짐
        if self.aggregator == 'sum':
            # self + neighbor 합산
            output = (self_vectors + neighbors_agg).view((-1, self.dim))

        elif self.aggregator == 'concat':
            # self와 neighbor를 concat한 후 projection
            output = torch.cat((self_vectors, neighbors_agg), dim=-1)  # [batch, -, 2*dim]
            output = output.view((-1, 2 * self.dim))

        else:
            # neighbor-only (self vector는 사용하지 않음)
            output = neighbors_agg.view((-1, self.dim))

        # projection + 활성화
        output = self.weights(output)  # [batch * -, dim]
        return act(output.view((self.batch_size, -1, self.dim)))  # 다시 배치 형태로 reshape

    def _mix_neighbor_vectors(self, neighbor_vectors, neighbor_relations, user_embeddings):
        # 사용자-관계에 따라 이웃 노드들을 가중 평균함 → user-aware attention으로 neighbor vector 집계
        # [batch, dim] → [batch, 1, 1, dim]
        user_embeddings = user_embeddings.view((self.batch_size, 1, 1, self.dim))

        # user와 relation 간의 점곱 score 계산 → [batch, -, n_neighbor]
        user_relation_scores = (user_embeddings * neighbor_relations).sum(dim=-1)

        # softmax로 attention weight 계산
        user_relation_scores_normalized = F.softmax(user_relation_scores, dim=-1)

        # [batch, -, n_neighbor] → [batch, -, n_neighbor, 1]
        user_relation_scores_normalized = user_relation_scores_normalized.unsqueeze(dim=-1)

        # attention weight * neighbor vector → 가중 평균 [batch, -, dim]
        neighbors_aggregated = (user_relation_scores_normalized * neighbor_vectors).sum(dim=2)

        return neighbors_aggregated

In [7]:
class KGCN(torch.nn.Module):
    def __init__(self, num_user, num_ent, num_rel, kg, config):
        super(KGCN, self).__init__()
        self.num_user = num_user
        self.num_ent = num_ent
        self.num_rel = num_rel
        self.n_iter = config['n_iter']                           # GCN 레이어 수
        self.batch_size = config['batch_size']                  # 배치 크기
        self.dim = config['embedding_dim']                      # 임베딩 차원
        self.n_neighbor = config['neighbor_sample_size']        # 이웃 수
        self.kg = kg
        self.device = torch.device(config['device'])            # device 설정

        # Aggregator 인스턴스
        self.aggregator = Aggregator(self.batch_size, self.dim, config['aggregator'])

        self._gen_adj()

        # 임베딩 레이어
        self.usr = torch.nn.Embedding(num_user, self.dim)
        self.ent = torch.nn.Embedding(num_ent, self.dim)
        self.rel = torch.nn.Embedding(num_rel, self.dim)

    def _gen_adj(self): # 엔티티마다 고정 개수의 이웃 엔티티 및 관계를 샘플링하여 인접행렬 생성
        self.adj_ent = torch.empty(self.num_ent, self.n_neighbor, dtype=torch.long)
        self.adj_rel = torch.empty(self.num_ent, self.n_neighbor, dtype=torch.long)

        for e in self.kg:
            if len(self.kg[e]) >= self.n_neighbor:
                neighbors = random.sample(self.kg[e], self.n_neighbor)
            else:
                neighbors = random.choices(self.kg[e], k=self.n_neighbor)  # 중복 허용 샘플링

            self.adj_ent[e] = torch.LongTensor([ent for _, ent in neighbors])
            self.adj_rel[e] = torch.LongTensor([rel for rel, _ in neighbors])

    def forward(self, u, v): # forward 호출 시 (user, item) 쌍 입력 → user-aware item score 출력 / u: [batch_size], v: [batch_size]
        batch_size = u.size(0)
        if batch_size != self.batch_size:
            self.batch_size = batch_size

        # shape 맞추기: [batch_size, 1]
        u = u.view((-1, 1))
        v = v.view((-1, 1))

        # 사용자 임베딩: [batch_size, dim]
        user_embeddings = self.usr(u).squeeze(dim=1)

        # item을 기준으로 multi-hop 이웃 엔티티/관계 가져오기
        entities, relations = self._get_neighbors(v)

        # 이웃 정보를 user-aware 방식으로 aggregation
        item_embeddings = self._aggregate(user_embeddings, entities, relations)

        # 최종 user-item score 계산: 내적 후 sigmoid
        scores = (user_embeddings * item_embeddings).sum(dim=1)

        return torch.sigmoid(scores)

    def _get_neighbors(self, v): # 엔티티 v의 multi-hop 이웃들을 adj matrix 기반으로 샘플링 / v: [batch_size, 1] → 1-hop, 2-hop ...까지 쌓임
        entities = [v]  # 0-hop (자기 자신)
        relations = []

        for h in range(self.n_iter):  # hop 수만큼 반복
            # 현재 hop의 엔티티에서 이웃 추출
            neighbor_entities = torch.LongTensor(self.adj_ent[entities[h].cpu()]) \
                                    .view((self.batch_size, -1)).to(self.device)
            neighbor_relations = torch.LongTensor(self.adj_rel[entities[h].cpu()]) \
                                    .view((self.batch_size, -1)).to(self.device)

            entities.append(neighbor_entities)
            relations.append(neighbor_relations)

        return entities, relations

    def _aggregate(self, user_embeddings, entities, relations): # Aggregator를 사용해 multi-hop 이웃 정보를 통합
        # user_embeddings: [batch_size, dim]
        # entities: hop별 entity 리스트
        # relations: hop별 relation 리스트
        
        # hop 별로 entity / relation 임베딩 추출
        entity_vectors = [self.ent(entity) for entity in entities]
        relation_vectors = [self.rel(relation) for relation in relations]

        # hop 수만큼 반복적으로 aggregation
        for i in range(self.n_iter):
            # 마지막 hop은 tanh, 그 외는 sigmoid
            act = torch.tanh if i == self.n_iter - 1 else torch.sigmoid

            entity_vectors_next_iter = []
            for hop in range(self.n_iter - i):
                vector = self.aggregator(
                    self_vectors=entity_vectors[hop],
                    neighbor_vectors=entity_vectors[hop + 1] \
                        .view((self.batch_size, -1, self.n_neighbor, self.dim)),
                    neighbor_relations=relation_vectors[hop] \
                        .view((self.batch_size, -1, self.n_neighbor, self.dim)),
                    user_embeddings=user_embeddings,
                    act=act
                )
                entity_vectors_next_iter.append(vector)

            # 다음 hop의 결과로 갱신
            entity_vectors = entity_vectors_next_iter

        # 최종 item 임베딩 반환
        return entity_vectors[0].view((self.batch_size, self.dim))

In [8]:
kg = data_loader.load_kg()
df_dataset

Construct knowledge graph ... Done


,userID,itemID,label
0,1217,289,1
1,1648,4018,0
2,596,8885,0
3,475,57,1
4,1450,5171,0
...,...,...,...
42341,1778,1663,0
42342,519,4,1
42343,1487,2879,1
42344,1115,36,1


In [9]:
# PyTorch Dataset 클래스 정의
class KGCNDataset(torch.utils.data.Dataset):
    def __init__(self, df): # df: 학습용 데이터프레임 (userID, itemID, label 포함)
        self.df = df

    def __len__(self): # 전체 샘플 개수 반환 → DataLoader에서 전체 데이터 크기 인식용
        return len(self.df)

    def __getitem__(self, idx): # 인덱스 idx에 해당하는 샘플을 반환 → DataLoader가 배치 단위로 호출할 때 사용됨
        user_id = np.array(self.df.iloc[idx]['userID']) # 사용자 ID
        item_id = np.array(self.df.iloc[idx]['itemID']) # 아이템 (entity) ID
        label = np.array(self.df.iloc[idx]['label'], dtype=np.float32) # 이진 라벨 (float32로 변환)

        return user_id, item_id, label

In [10]:
# 텐서 데이터셋 구성
train_data = TensorDataset(
    torch.LongTensor(df_dataset['userID'].values),
    torch.LongTensor(df_dataset['itemID'].values),
    torch.FloatTensor(df_dataset['label'].values)
)

train_loader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)

# 모델 생성에 필요한 수치 정보
n_user, n_entity, n_relation = data_loader.get_num()
kg_dict = data_loader.load_kg()

# 모델 생성 (config 통합 버전의 KGCN 클래스 사용)
model = KGCN(num_user=n_user, num_ent=n_entity, num_rel=n_relation, kg=kg_dict, config=config).to(device)

print("모델 및 데이터셋 준비 완료")

Construct knowledge graph ... Done
모델 및 데이터셋 준비 완료


In [11]:
# 데이터 분할 (config['train_ratio'] 사용)
x_train, x_test, y_train, y_test = train_test_split(
    df_dataset,
    df_dataset['label'],
    test_size=1 - config['train_ratio'],
    shuffle=False,               # 시계열 데이터면 False, 랜덤이면 True
    random_state=999             # 재현성 보장을 위한 고정 seed
)

# 커스텀 Dataset 클래스 활용
train_dataset = KGCNDataset(x_train)
test_dataset = KGCNDataset(x_test)

# DataLoader 구성 (config 사용)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

In [12]:
# 사용자, 엔티티, 관계 수 얻기
num_user, num_entity, num_relation = data_loader.get_num()

# 인코더들 (원한다면 추후 디코딩에 사용 가능)
user_encoder, entity_encoder, relation_encoder = data_loader.get_encoders()

# KGCN 모델 생성 (config 기반)
net = KGCN(num_user, num_entity, num_relation, kg_dict, config).to(device)

# Binary Cross Entropy Loss (sigmoid + float label에 적합)
criterion = torch.nn.BCELoss()

# Adam 옵티마이저 (학습률 + L2 정규화는 config에서)
optimizer = optim.Adam(
    net.parameters(),
    lr=config['lr'],
    weight_decay=config['l2_weight']
)

print('device:', device)


device: mps


In [13]:
loss_list = []
test_loss_list = []
auc_score_list = []

# 학습 에폭 반복
for epoch in range(config['n_epochs']):
    net.train()  # 학습 모드
    running_loss = 0.0

    # 1 epoch 동안 전체 배치 학습
    for user_ids, item_ids, labels in train_loader:
        user_ids, item_ids, labels = user_ids.to(device), item_ids.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(user_ids, item_ids)  # forward
        loss = criterion(outputs, labels)  # BCE loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # 에폭별 평균 학습 손실 저장 및 출력
    avg_train_loss = running_loss / len(train_loader)
    loss_list.append(avg_train_loss)
    print(f'[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f}')

    net.eval()
    with torch.no_grad():
        test_loss = 0
        total_auc = 0

        for user_ids, item_ids, labels in test_loader:
            user_ids, item_ids, labels = user_ids.to(device), item_ids.to(device), labels.to(device)

            outputs = net(user_ids, item_ids)
            test_loss += criterion(outputs, labels).item()

            # AUC 계산 (label, 예측 확률)
            total_auc += roc_auc_score(labels.cpu().numpy(), outputs.cpu().numpy())

        avg_test_loss = test_loss / len(test_loader)
        avg_auc = total_auc / len(test_loader)

        test_loss_list.append(avg_test_loss)
        auc_score_list.append(avg_auc)

        print(f'[Epoch {epoch+1}] Test Loss: {avg_test_loss:.4f} | AUC: {avg_auc:.4f}')

[Epoch 1] Train Loss: 1.0593
[Epoch 1] Test Loss: 1.0450 | AUC: 0.4985
[Epoch 2] Train Loss: 0.9810
[Epoch 2] Test Loss: 0.9757 | AUC: 0.4980
[Epoch 3] Train Loss: 0.9045
[Epoch 3] Test Loss: 0.9077 | AUC: 0.4978
[Epoch 4] Train Loss: 0.8360
[Epoch 4] Test Loss: 0.8460 | AUC: 0.4967
[Epoch 5] Train Loss: 0.7820
[Epoch 5] Test Loss: 0.7957 | AUC: 0.4950
[Epoch 6] Train Loss: 0.7414
[Epoch 6] Test Loss: 0.7594 | AUC: 0.4933
[Epoch 7] Train Loss: 0.7163
[Epoch 7] Test Loss: 0.7357 | AUC: 0.4906
[Epoch 8] Train Loss: 0.7017
[Epoch 8] Test Loss: 0.7219 | AUC: 0.4883
[Epoch 9] Train Loss: 0.6943
[Epoch 9] Test Loss: 0.7140 | AUC: 0.4879
[Epoch 10] Train Loss: 0.6903
[Epoch 10] Test Loss: 0.7094 | AUC: 0.4864
[Epoch 11] Train Loss: 0.6884
[Epoch 11] Test Loss: 0.7067 | AUC: 0.4868
[Epoch 12] Train Loss: 0.6874
[Epoch 12] Test Loss: 0.7048 | AUC: 0.4870
[Epoch 13] Train Loss: 0.6866
[Epoch 13] Test Loss: 0.7039 | AUC: 0.4859
[Epoch 14] Train Loss: 0.6859
[Epoch 14] Test Loss: 0.7030 | AUC: 0.4